# LLY-DML: Quantum Circuit Training mit L-Gates

<div style="text-align: center; padding: 10px; margin: 10px;">
    <h3>LILY Quantum Machine Learning - Differentiable Machine Learning</h3>
    <p style="font-style: italic;">Ein Tutorial zur Quantenoptimierung mit L-Gates</p>
</div>

---

## Installation der benötigten Pakete

Da wir dieses Notebook in Google Colab ausführen, installieren wir zunächst alle notwendigen Pakete.

In [ ]:
# Installation der benötigten Bibliotheken
!pip install qiskit matplotlib numpy seaborn

## 1. Einführung zu L-Gates

Die L-Gate-Struktur ist das Grundelement unserer quantum-basierten Lernarchitektur. Ein L-Gate besteht aus einer spezifischen Abfolge von Quantengates:

```
TP0 → IP0 → H → TP1 → IP1 → H → TP2 → IP2
```

Wobei:
- **TP** = Trainingsphase (P-Gate mit trainierbare Parameter)
- **IP** = Inputphase (P-Gate mit feste Inputparameter)
- **H** = Hadamard-Gate (festes Gate ohne Parameter)

Diese Struktur wird für jeden Qubit und in der festgelegten Tiefe (Depth) wiederholt. Die L-Gate-Struktur ermöglicht es uns, einen Quantum Circuit zu trainieren, sodass er verschiedene Eingaben (Inputmatrizen) auf unterschiedliche Quantenzustände abbildet.

In [ ]:
# Basisbibliotheken importieren
import sys
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import logging

# Qiskit für Quantum Circuit Simulation
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("DML-Tutorial")

# Matplotlib-Stil setzen
plt.style.use('default')
sns.set_style("whitegrid")
plt.rcParams.update({'font.size': 14})

## 2. Visualisierung eines L-Gates

Lass uns ein einzelnes L-Gate visualisieren, um seine Struktur besser zu verstehen.

In [ ]:
# Erstelle einen einfachen L-Gate-Circuit für die Visualisierung
l_gate_circuit = QuantumCircuit(1)

# Füge ein L-Gate hinzu
# TP0 → IP0 → H → TP1 → IP1 → H → TP2 → IP2
l_gate_circuit.p(0.1, 0)  # TP0 - Trainingsphase 0 (trainierbar)
l_gate_circuit.p(0.2, 0)  # IP0 - Inputphase 0 (fest)
l_gate_circuit.h(0)       # H   - Hadamard-Gate
l_gate_circuit.p(0.3, 0)  # TP1 - Trainingsphase 1 (trainierbar)
l_gate_circuit.p(0.4, 0)  # IP1 - Inputphase 1 (fest)
l_gate_circuit.h(0)       # H   - Hadamard-Gate
l_gate_circuit.p(0.5, 0)  # TP2 - Trainingsphase 2 (trainierbar)
l_gate_circuit.p(0.6, 0)  # IP2 - Inputphase 2 (fest)

# Visualisiere den L-Gate-Circuit
plt.figure(figsize=(16, 4))
circuit_drawer(l_gate_circuit, output='mpl', style={"name": "bw", "cregbundle": False})
plt.title("Struktur eines einzelnen L-Gates", fontsize=16)
plt.tight_layout()
plt.show()

# Erläuterung des L-Gates
print("L-Gate-Struktur besteht aus 8 Gates in dieser Reihenfolge:")
print("1. TP0 (P-Gate): Trainingsphase 0 - trainierbar")
print("2. IP0 (P-Gate): Inputphase 0 - fester Input")
print("3. H (Hadamard): Superposition erzeugen")
print("4. TP1 (P-Gate): Trainingsphase 1 - trainierbar")
print("5. IP1 (P-Gate): Inputphase 1 - fester Input")
print("6. H (Hadamard): Superposition erzeugen")
print("7. TP2 (P-Gate): Trainingsphase 2 - trainierbar")
print("8. IP2 (P-Gate): Inputphase 2 - fester Input")

## 3. Implementierung des LLY-DML Circuit-Systems

Da wir in Google Colab arbeiten und keinen direkten Zugriff auf das LLY-DML-Framework haben, implementieren wir hier eine vereinfachte Version der Circuit- und Optimizer-Klassen.

In [ ]:
# Implementierung des Circuit-Systems
class Circuit:
    """
    Vereinfachte Implementierung der LLY-DML Circuit-Klasse für dieses Tutorial.
    """
    def __init__(self, qubits, depth, training_phases=None, activation_phases=None, shots=1024):
        """
        Initialisiert einen Quantum Circuit mit angegebener Qubit-Anzahl und Tiefe.
        
        Args:
            qubits (int): Anzahl der Qubits im Circuit
            depth (int): Anzahl der L-Gates pro Qubit
            training_phases (list, optional): 3D-Matrix mit Trainingsparametern [qubits × depth × 3]
            activation_phases (list, optional): 3D-Matrix mit Aktivierungsparametern [qubits × depth × 3]
            shots (int, optional): Anzahl der Messungen bei der Ausführung
        """
        self.qubits = qubits
        self.depth = depth
        self.shots = shots
        self.simulation_result = None
        
        # Erstelle die Circuit-Struktur
        self.circuit = None
        self.build_circuit()
        
        # Initialisiere Trainings- und Aktivierungsmatrizen wenn nicht angegeben
        if training_phases is None:
            self.training_phases = self.create_matrix()
        else:
            self.training_phases = training_phases
            
        if activation_phases is None:
            self.activation_phases = self.create_matrix()
        else:
            self.activation_phases = activation_phases
            
        # Platziere Gates im Circuit
        self.place_matrices()
    
    def build_circuit(self):
        """
        Erstellt einen leeren Quantum Circuit mit der angegebenen Anzahl von Qubits.
        """
        from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
        q_register = QuantumRegister(self.qubits, 'q')
        c_register = ClassicalRegister(self.qubits, 'c')
        self.circuit = QuantumCircuit(q_register, c_register)
    
    def create_matrix(self):
        """
        Erstellt eine Matrix mit zufälligen Werten zwischen 0 und 2π.
        
        Returns:
            list: 3D-Matrix mit zufälligen Werten [qubits × depth × 3]
        """
        matrix = np.random.random((self.qubits, self.depth, 3)) * 2 * np.pi
        return matrix.tolist()
    
    def place_matrices(self):
        """
        Überträgt die Trainings- und Aktivierungsdaten aus den Matrizen in den Quantum Circuit.
        """
        # Validiere die Dimensionen der Matrizen
        if len(self.training_phases) != self.qubits or len(self.activation_phases) != self.qubits:
            raise ValueError(f"Training and activation phases must each have {self.qubits} rows.")
            
        # Platziere L-Gates für jedes Qubit
        for qubit in range(self.qubits):
            for d in range(self.depth):
                # Ein L-Gate besteht aus 3 Phasen mit jeweils Trainings- und Aktivierungsphase
                for phase in range(3):
                    # Index in der Trainings-/Aktivierungsmatrix
                    phase_index = phase
                    
                    # Trainingsphase mit Phasengates (p)
                    self.circuit.p(self.training_phases[qubit][d][phase_index], qubit)
                    self.circuit.p(self.activation_phases[qubit][d][phase_index], qubit)
                    
                    # Hadamard nach Phase 0 und 1
                    if phase == 0 or phase == 1:
                        self.circuit.h(qubit)
    
    def place_measurement(self):
        """
        Fügt Messgates für alle Qubits am Ende des Circuits hinzu.
        """
        self.circuit.measure(range(self.qubits), range(self.qubits))
    
    def plot_circuit(self):
        """
        Zeichnet den aktuellen Quantum Circuit.
        """
        from qiskit.visualization import circuit_drawer
        return circuit_drawer(self.circuit, output='mpl', style={'name': 'bw'})
    
    def run(self):
        """
        Führt den Quantum Circuit auf dem Simulator aus.
        """
        from qiskit import Aer, transpile
        
        # Füge Messungen hinzu, falls noch nicht vorhanden
        if not hasattr(self.circuit, 'cregs') or not self.circuit.cregs:
            self.place_measurement()
            
        simulator = Aer.get_backend('aer_simulator')
        compiled_circuit = transpile(self.circuit, simulator)
        self.simulation_result = simulator.run(compiled_circuit, shots=self.shots).result()
        return self.simulation_result
    
    def get_counts(self):
        """
        Gibt die Messungsergebnisse als Dictionary zurück.
        """
        if self.simulation_result is None:
            raise RuntimeError("The circuit has not been executed yet.")
        return self.simulation_result.get_counts(self.circuit)
    
    def get_state_probabilities(self, counts=None):
        """
        Berechnet die Wahrscheinlichkeiten der gemessenen Zustände.
        """
        if counts is None:
            if self.simulation_result is None:
                raise RuntimeError("The circuit has not been executed yet.")
            counts = self.get_counts()
            
        total_shots = sum(counts.values())
        return {state: count/total_shots for state, count in counts.items()}
    
    def set_train_parameters(self, parameters):
        """
        Setzt die Trainingsparameter.
        """
        if isinstance(parameters, np.ndarray):
            self.training_phases = parameters.tolist()
        else:
            self.training_phases = parameters
        return self.training_phases
    
    def place_train_matrix(self):
        """
        Aktualisiert den Circuit mit den aktuellen Trainingsparametern.
        """
        # Erstelle einen neuen Circuit
        self.build_circuit()
        # Platziere die Matrizen
        self.place_matrices()
        return self.circuit
    
    def place_input_parameters(self, input_data_matrix):
        """
        Setzt die Inputparameter und aktualisiert den Circuit.
        """
        if isinstance(input_data_matrix, np.ndarray):
            self.activation_phases = input_data_matrix.tolist()
        else:
            self.activation_phases = input_data_matrix
            
        # Aktualisiere den Circuit
        self.build_circuit()
        self.place_matrices()
        return self.circuit

In [ ]:
# Implementierung des Adam-Optimizers
class AdamOptimizer:
    """
    Vereinfachte Implementierung des Adam-Optimizers für dieses Tutorial.
    """
    def __init__(self, data, training_matrix, target_state, learning_rate=0.001, max_iterations=100, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.data = data
        self.training_matrix = training_matrix
        self.target_state = target_state
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        
        # Initialisiere tuning_parameters mit der Trainingsmatrix
        self.tuning_parameters = np.array(training_matrix)
        # Initialisiere die Momentschätzer
        self.m = np.zeros_like(self.tuning_parameters)
        self.v = np.zeros_like(self.tuning_parameters)
    
    def evaluate(self):
        """
        Bewertet die aktuelle Matrix im Vergleich zum Zielzustand.
        """
        return self.calculate_loss()
    
    def calculate_loss(self):
        """
        Berechnet den Verlust basierend auf der Trainingsmatrix und dem Zielzustand.
        """
        # Verlustfunktion basierend auf dem Zielzustand (ein einzelnes Bit '0' oder '1')
        loss = 0
        matrix_flat = self.tuning_parameters.flatten()
        
        # Das Zielbit als Integer konvertieren (0 oder 1)
        desired_bit = int(self.target_state)
        
        # Für jeden Parameter in der flachen Matrix
        for param in matrix_flat:
            # Normalisiere den Parameter auf [0, 1]
            normalized_param = (np.sin(param) + 1) / 2
            
            # Abstand zum gewünschten Bit berechnen
            if desired_bit == 0:
                loss += normalized_param  # Wenn 0 gewünscht, sollte der Parameter klein sein
            else:
                loss += (1 - normalized_param)  # Wenn 1 gewünscht, sollte der Parameter groß sein
        
        # Der durchschnittliche Verlust über alle Parameter
        loss = loss / len(matrix_flat) if len(matrix_flat) > 0 else 0
        return loss
    
    def calculate_loss_specific(self, params):
        """
        Berechnet den Verlust für spezifische Parameter.
        """
        # Ähnliche Implementierung wie calculate_loss, aber mit übergebenen Parametern
        loss = 0
        desired_bit = int(self.target_state)
        
        for param in params:
            normalized_param = (np.sin(param) + 1) / 2
            
            if desired_bit == 0:
                loss += normalized_param
            else:
                loss += (1 - normalized_param)
        
        loss = loss / len(params) if len(params) > 0 else 0
        return loss
    
    def compute_gradient(self):
        """
        Berechnet den Gradienten der Verlustfunktion nach den Trainingsparametern.
        """
        params = self.tuning_parameters.flatten()
        gradients = np.zeros_like(params)
        
        epsilon = 1e-5  # Kleine Änderung zur Berechnung der numerischen Ableitung
        
        for i in range(len(params)):
            # Temporäre Parameter, um den Einfluss von Parameter i zu bewerten
            params_plus = params.copy()
            params_plus[i] += epsilon
            
            params_minus = params.copy()
            params_minus[i] -= epsilon
            
            # Berechne den Gradienten mittels zentraler Differenz
            loss_plus = self.calculate_loss_specific(params_plus)
            loss_minus = self.calculate_loss_specific(params_minus)
            
            gradients[i] = (loss_plus - loss_minus) / (2 * epsilon)
        
        return gradients.reshape(self.tuning_parameters.shape)
    
    def optimize(self):
        """
        Führt den Adam-Optimierungsprozess durch.
        """
        logging.info("AdamOptimizer: Startet den Optimierungsprozess")
        optimization_steps = []
        
        for iteration in range(1, self.max_iterations + 1):
            # Berechne den aktuellen Verlust
            loss = self.evaluate()
            optimization_steps.append({"iteration": iteration, "loss": loss})
            
            # Frühzeitiger Abbruch bei minimalem Verlust
            if loss == 0:
                logging.info("AdamOptimizer: Verlust unter dem Schwellenwert, Optimierung abgeschlossen")
                break
            
            # Berechne den Gradienten
            gradient = self.compute_gradient()
            
            # Adam-Optimierungsschritt
            self.m = self.beta1 * self.m + (1 - self.beta1) * gradient
            self.v = self.beta2 * self.v + (1 - self.beta2) * (gradient ** 2)
            
            # Bias-Korrektur
            m_hat = self.m / (1 - self.beta1 ** iteration)
            v_hat = self.v / (1 - self.beta2 ** iteration)
            
            # Update der Parameter
            update = self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)
            self.tuning_parameters -= update
        
        logging.info("AdamOptimizer: Optimierungsprozess abgeschlossen")
        return self.tuning_parameters.tolist(), optimization_steps

## 4. Ein einfaches Beispiel: Einzelne Eingabematrix und Circuit-Ausführung

Bevor wir mit der Multi-Matrix-Optimierung beginnen, untersuchen wir, wie eine einzelne Eingabematrix in den Circuit eingesetzt wird und wie der Circuit aussieht.

In [ ]:
# Parameter für den Beispiel-Circuit
qubits = 5
depth = 3
shots = 1024

# Erstelle eine Eingabematrix
input_matrix = np.random.uniform(0, 2*np.pi, (qubits, depth, 3))
print(f"Eingabematrix-Form: {input_matrix.shape}")

# Visualisiere die Eingabematrix als Heatmap
plt.figure(figsize=(12, 6))
input_flat = input_matrix.reshape(qubits, -1)
sns.heatmap(input_flat, cmap="viridis", vmin=0, vmax=2*np.pi)
plt.title("Beispiel-Eingabematrix - Werte zwischen 0 und 2π")
plt.xlabel("Parameter (flach: Tiefe × 3 Phasen)")
plt.ylabel("Qubit")
plt.tight_layout()
plt.show()

# Erstelle die Trainingsmatrix
training_matrix = np.zeros((qubits, depth, 3))
print(f"Trainingsmatrix-Form: {training_matrix.shape}")

# Erstelle einen Circuit und setze die Matrix ein
circuit = Circuit(qubits=qubits, depth=depth, 
                 training_phases=training_matrix.tolist(),
                 activation_phases=input_matrix.tolist(),
                 shots=shots)

# Visualisiere den Circuit (begrenzt auf 2 Qubits und 1 L-Gate für die Übersichtlichkeit)
smaller_circuit = Circuit(qubits=2, depth=1,
                         training_phases=training_matrix[:2, :1, :].tolist(),
                         activation_phases=input_matrix[:2, :1, :].tolist())
smaller_circuit.place_measurement()

plt.figure(figsize=(16, 6))
smaller_circuit.plot_circuit()
plt.title("Vereinfachter Circuit mit L-Gates (2 Qubits, 1 L-Gate)", fontsize=16)
plt.tight_layout()
plt.show()

# Beschreibung der Matrixeinbettung
print("Einbettung der Matrizen in den Circuit:")
print("1. Jedes Qubit bekommt eine Zeile aus der Matrix")
print("2. Jedes L-Gate verwendet 3 Werte aus jeder Matrix (Training und Input)")
print("3. Die Trainingsmatrix liefert die TP-Phasen (trainierbar)")
print("4. Die Eingabematrix liefert die IP-Phasen (fest)")

## 5. Multi-Matrix-Klassifikation: Training und Optimierung

Jetzt kommen wir zum Hauptteil: Der Optimierung eines Circuits, um verschiedene Eingabematrizen auf unterschiedliche Zielzustände abzubilden. Dies entspricht einer Klassifikationsaufgabe in der Quantenwelt.

In [ ]:
# Parameter für die Multi-Matrix-Optimierung
qubits = 5
depth = 3
shots = 1024
num_matrices = 4  # Anzahl der Eingabematrizen

# Erstelle mehrere Eingabematrizen
input_matrices = []
matrix_names = []

for i in range(num_matrices):
    # Verwende unterschiedliche Bereiche für verschiedene Matrizen
    matrix = np.random.uniform(i*0.5, i*0.5 + 2*np.pi, (qubits, depth, 3))
    input_matrices.append(matrix)
    matrix_names.append(f"Matrix_{i+1}")

# Definiere Zielzustände für jede Matrix
target_states = {}
for i, name in enumerate(matrix_names):
    # Erzeuge einen binären Zustand mit der richtigen Länge
    binary = format(i+1, f'0{qubits}b')
    # Sichere die richtige Länge (gleich qubits)
    target_states[name] = binary[-qubits:]

# Zeige die zugewiesenen Zielzustände
print("Zugewiesene Zielzustände für jede Matrix:")
print("-" * 50)
print(f"{'Matrix':10s} | {'Zielzustand':12s}")
print("-" * 50)
for name, state in target_states.items():
    print(f"{name:10s} | {state:12s}")
print("-" * 50)

### Initiale Messungen

Bevor wir mit der Optimierung beginnen, messen wir die anfänglichen Wahrscheinlichkeiten für jede Matrix.

In [ ]:
# Initialisiere die gemeinsame Trainingsmatrix (zunächst alle Parameter 0)
training_matrix = np.zeros((qubits, depth, 3))

# Initiale Messungen für jede Matrix
initial_measurements = []

for i, (matrix, name) in enumerate(zip(input_matrices, matrix_names)):
    target_state = target_states[name]
    
    # Erstelle einen Circuit mit der Trainings- und Eingabematrix
    circuit = Circuit(qubits, depth, 
                      training_phases=training_matrix.tolist(),
                      activation_phases=matrix.tolist())
    
    # Füge Messungen hinzu und führe den Circuit aus
    circuit.place_measurement()
    circuit.run()
    counts = circuit.get_counts()
    
    # Berechne die Wahrscheinlichkeiten
    probabilities = circuit.get_state_probabilities(counts)
    
    # Bestimme den wahrscheinlichsten Zustand
    sorted_states = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)
    most_probable_state = sorted_states[0][0] if sorted_states else "N/A"
    most_probable_prob = sorted_states[0][1] if sorted_states else 0
    
    # Berechne die Wahrscheinlichkeit des Zielzustands
    target_probability = probabilities.get(target_state, 0)
    
    # Speichere die Messung
    initial_measurements.append({
        "matrix_name": name,
        "target_state": target_state,
        "most_probable_state": most_probable_state,
        "most_probable_prob": most_probable_prob,
        "target_probability": target_probability
    })

# Zeige die initialen Messungen
print("Initiale Messungen vor der Optimierung:")
print("-" * 80)
print(f"{'Matrix':10s} | {'Zielzustand':12s} | {'Wahrscheinlichster':15s} | {'Wahrsch.':8s} | {'Ziel-Wahrsch.':12s}")
print("-" * 80)
for result in initial_measurements:
    print(f"{result['matrix_name']:10s} | {result['target_state']:12s} | "
          f"{result['most_probable_state']:15s} | {result['most_probable_prob']:.4f} | "
          f"{result['target_probability']:.4f}")
print("-" * 80)

### Optimierungsfunktion

Implementieren wir eine Funktion, die die Trainingsmatrix für einen bestimmten Zielzustand optimiert.

In [ ]:
def optimize_for_target_state(input_matrix, training_matrix, target_state, iterations=20):
    """
    Optimiert eine Trainingsmatrix für eine Eingabematrix und einen Zielzustand.
    
    Args:
        input_matrix: Die Eingabematrix für den Circuit
        training_matrix: Die initiale Trainingsmatrix
        target_state: Der Zielzustand als binärer String
        iterations: Anzahl der Trainingsiterationen
    
    Returns:
        tuple: (optimierte Trainingsmatrix, Trainingsgeschichte)
    """
    # Optimizer-Konfiguration
    optimizer_data = {"qubits": qubits, "depth": depth}
    
    # Verwende das erste Bit des Zielzustands für den Optimizer
    # (Vereinfachung für das Tutorial)
    first_bit = target_state[0]
    
    # Adam-Optimizer initialisieren
    optimizer = AdamOptimizer(
        data=optimizer_data,
        training_matrix=training_matrix.flatten().tolist(),
        target_state=first_bit,
        learning_rate=0.01,
        max_iterations=iterations
    )
    
    # Trainingsgeschichte initialisieren
    history = []
    current_matrix = training_matrix.copy()
    
    # Circuit für die Optimierung
    circuit = Circuit(qubits, depth)
    
    # Training über mehrere Iterationen
    for iteration in range(iterations):
        # Aktualisiere den Circuit mit den aktuellen Matrizen
        circuit.place_input_parameters(input_matrix)
        circuit.set_train_parameters(current_matrix)
        circuit.place_train_matrix()
        
        # Messung hinzufügen und Circuit ausführen
        circuit.place_measurement()
        circuit.run()
        counts = circuit.get_counts()
        
        # Optimierungsschritt durchführen
        try:
            optimized_params, opt_steps = optimizer.optimize()
            
            # Reshape der optimierten Parameter
            if len(optimized_params) == qubits * depth * 3:
                reshaped_params = np.array(optimized_params).reshape(qubits, depth, 3)
                current_matrix = reshaped_params
            else:
                logger.warning(f"Optimierte Parameter haben falsche Größe: {len(optimized_params)}")
        except Exception as e:
            logger.error(f"Fehler bei der Optimierung: {e}")
            continue
        
        # Wahrscheinlichkeit des Zielzustands berechnen
        probabilities = circuit.get_state_probabilities(counts)
        target_prob = probabilities.get(target_state, 0.0)
        
        # Verlust aus dem letzten Optimierungsschritt
        loss = opt_steps[-1]["loss"] if opt_steps else 1.0
        
        # Trainingsschritt speichern
        history.append({
            "iteration": iteration + 1,
            "target_probability": target_prob,
            "loss": loss
        })
        
        # Status ausgeben
        if (iteration + 1) % 5 == 0 or iteration == 0:
            logger.info(f"Iteration {iteration+1}/{iterations}, "
                        f"Zielwahrscheinlichkeit: {target_prob:.4f}, Loss: {loss:.4f}")
    
    return current_matrix, history

### Durchführung der Optimierung

Jetzt führen wir die Optimierung für alle Eingabematrizen sequentiell durch.

In [ ]:
# Optimierungsparameter
training_iterations = 20

# Ergebnisse speichern
optimization_results = {
    "initial_states": {},
    "final_states": {},
    "training_history": {}
}

# Initialisiere die gemeinsame Trainingsmatrix (Anfangswerte nahe Null)
current_training_matrix = np.random.uniform(0, np.pi/8, (qubits, depth, 3))

# Optimiere für jede Eingabematrix
for i, (matrix, name) in enumerate(zip(input_matrices, matrix_names)):
    target_state = target_states[name]
    
    print(f"\n{'='*50}")
    print(f"Optimierung für {name} mit Zielzustand '{target_state}'")
    print(f"{'='*50}")
    
    # Führe die Optimierung durch
    optimized_matrix, history = optimize_for_target_state(
        input_matrix=matrix,
        training_matrix=current_training_matrix,
        target_state=target_state,
        iterations=training_iterations
    )
    
    # Aktualisiere die Trainingsmatrix für die nächste Eingabematrix
    current_training_matrix = optimized_matrix
    
    # Speichere die Ergebnisse
    optimization_results["initial_states"][i] = {
        "name": name,
        "state": target_state,
        "probability": initial_measurements[i]["target_probability"]
    }
    
    # Berechne die finale Wahrscheinlichkeit
    final_prob = history[-1]["target_probability"] if history else 0.0
    optimization_results["final_states"][i] = {
        "name": name,
        "state": target_state,
        "probability": final_prob,
        "loss": history[-1]["loss"] if history else 1.0
    }
    
    optimization_results["training_history"][i] = history
    
    print(f"Optimierung abgeschlossen. Finale Wahrscheinlichkeit für '{target_state}': {final_prob:.4f}")

## 6. Visualisierung des Trainingsfortschritts

Lassen Sie uns den Trainingsfortschritt für jede Matrix visualisieren.

In [ ]:
# Trainingsfortschritt visualisieren
plt.figure(figsize=(14, 6))

for matrix_idx, history in optimization_results["training_history"].items():
    iterations = [entry["iteration"] for entry in history]
    probabilities = [entry["target_probability"] for entry in history]
    
    matrix_name = optimization_results["initial_states"][matrix_idx]["name"]
    target_state = optimization_results["initial_states"][matrix_idx]["state"]
    
    plt.plot(iterations, probabilities, marker='o', linewidth=2, 
             label=f"{matrix_name} → '{target_state}'")

plt.title("Trainingsfortschritt - Wahrscheinlichkeit des Zielzustands", fontsize=16)
plt.xlabel("Iteration")
plt.ylabel("Wahrscheinlichkeit")
plt.grid(True, alpha=0.3)
plt.legend()
plt.ylim(0, 1.0)
plt.tight_layout()
plt.show()

## 7. Finale Validierung

Jetzt validieren wir die optimierte Trainingsmatrix mit jeder Eingabematrix.

In [ ]:
# Validierung mit der optimierten Trainingsmatrix
validation_results = []

for i, (matrix, name) in enumerate(zip(input_matrices, matrix_names)):
    target_state = target_states[name]
    
    # Erstelle einen neuen Circuit mit der optimierten Trainingsmatrix und der aktuellen Eingabematrix
    circuit = Circuit(qubits, depth, 
                      training_phases=current_training_matrix.tolist(),
                      activation_phases=matrix.tolist(), 
                      shots=1024)
    
    # Führe den Circuit aus und erhalte die Counts
    circuit.place_measurement()
    circuit.run()
    counts = circuit.get_counts()
    
    # Berechne die Wahrscheinlichkeiten
    probabilities = circuit.get_state_probabilities(counts)
    
    # Ermittle den wahrscheinlichsten Zustand
    sorted_states = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)
    most_probable_state = sorted_states[0][0] if sorted_states else "N/A"
    most_probable_prob = sorted_states[0][1] if sorted_states else 0
    target_probability = probabilities.get(target_state, 0)
    
    # Speichere die Ergebnisse
    validation_results.append({
        "matrix_name": name,
        "target_state": target_state,
        "most_probable_state": most_probable_state,
        "most_probable_prob": most_probable_prob,
        "target_probability": target_probability,
        "is_correct": most_probable_state == target_state,
        "initial_probability": initial_measurements[i]["target_probability"]
    })

# Vergleichstabelle: Anfangs- vs. Endwahrscheinlichkeiten
print("Vergleich der Wahrscheinlichkeiten vor und nach der Optimierung:")
print("-" * 90)
print(f"{'Matrix':10s} | {'Zielzustand':12s} | {'Anfangs-W.':10s} | {'End-W.':10s} | {'Änderung':10s} | {'Korrekt':8s}")
print("-" * 90)

for result in validation_results:
    initial_prob = result["initial_probability"]
    final_prob = result["target_probability"]
    change = final_prob - initial_prob
    change_str = f"{change:+.4f}"
    
    print(f"{result['matrix_name']:10s} | {result['target_state']:12s} | "
          f"{initial_prob:.4f} | {final_prob:.4f} | {change_str} | "
          f"{('✓' if result['is_correct'] else '✗'):8s}")

print("-" * 90)
print(f"Korrekte Klassifikationen: {sum(1 for r in validation_results if r['is_correct'])} von {len(validation_results)}")

## 8. Abschließende Visualisierung

Erstellen wir eine visuelle Zusammenfassung der Optimierungsergebnisse.

In [ ]:
# Visualisierung der Verbesserung für jede Matrix
plt.figure(figsize=(14, 6))

matrix_names = [r["matrix_name"] for r in validation_results]
initial_probs = [r["initial_probability"] for r in validation_results]
final_probs = [r["target_probability"] for r in validation_results]

x = np.arange(len(matrix_names))
width = 0.35

plt.bar(x - width/2, initial_probs, width, label='Vor Optimierung', color='skyblue')
plt.bar(x + width/2, final_probs, width, label='Nach Optimierung', color='coral')

plt.title("Verbesserung der Zielzustandswahrscheinlichkeit durch Optimierung", fontsize=16)
plt.xlabel("Eingabematrix")
plt.ylabel("Wahrscheinlichkeit")
plt.xticks(x, [f"{name}\n'{validation_results[i]['target_state']}'" 
               for i, name in enumerate(matrix_names)])
plt.grid(True, axis='y', alpha=0.3)
plt.ylim(0, 1.0)
plt.legend()

# Füge Werte über den Balken hinzu
for i, (initial, final) in enumerate(zip(initial_probs, final_probs)):
    plt.text(i - width/2, initial + 0.02, f"{initial:.3f}", ha='center')
    plt.text(i + width/2, final + 0.02, f"{final:.3f}", ha='center')
    # Verbesserung anzeigen
    change = final - initial
    color = 'green' if change > 0 else 'red'
    plt.text(i, max(initial, final) + 0.1, f"{change:+.3f}", ha='center', 
             color=color, fontweight='bold')

plt.tight_layout()
plt.show()

# Heatmap der optimierten Trainingsmatrix
plt.figure(figsize=(12, 6))
optimized_flat = current_training_matrix.reshape(qubits, -1)
sns.heatmap(optimized_flat, cmap="viridis", vmin=0, vmax=2*np.pi)
plt.title("Optimierte Trainingsmatrix", fontsize=16)
plt.xlabel("Parameter (flach: Tiefe × 3 Phasen)")
plt.ylabel("Qubit")
plt.tight_layout()
plt.show()

## 9. Zusammenfassung und Schlussfolgerungen

In diesem Tutorial haben wir gesehen, wie mit L-Gates in Quantum Circuits eine Klassifikationsaufgabe gelöst werden kann:

1. **L-Gate-Struktur**: Wir haben die L-Gate-Struktur kennengelernt, die aus einer spezifischen Abfolge von Trainingsphasen (TP), Inputphasen (IP) und Hadamard-Gates (H) besteht.

2. **Eingabematrizen**: Wir haben verschiedene Eingabematrizen erstellt und jeder einen eindeutigen Zielzustand zugewiesen.

3. **Optimierung**: Mit dem Adam-Optimierer haben wir die Trainingsparameter optimiert, damit der Circuit bei verschiedenen Eingaben die zugehörigen Zielzustände mit höherer Wahrscheinlichkeit liefert.

4. **Evaluation**: Wir haben den Erfolg der Optimierung gemessen, indem wir die Wahrscheinlichkeiten der Zielzustände vor und nach dem Training verglichen haben.

Diese Methode demonstriert, wie Quantum Circuits als Klassifikatoren eingesetzt werden können und wie durch gezielte Optimierung der Trainingsparameter die gewünschten Zuordnungen erreicht werden.

In realen Anwendungen könnte dieser Ansatz für Mustererkennung, Datenklassifikation oder sogar für maschinelles Lernen auf Quantencomputern genutzt werden.